In [123]:
import pandas as pd
import numpy as np
import ast

In [124]:
df = pd.read_csv('/Users/kilian/Documents/GitHub/The-Rockommendation/P2_G5_films.csv.gz', compression = 'gzip')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_12443/3721590168.py:1: DtypeWarning: Columns (0,1,4,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/kilian/Documents/GitHub/The-Rockommendation/P2_G5_films.csv.gz', compression = 'gzip')
